I will use the **pairs.csv** file to perform my pair analysis, but the file is organized in a way that is not easy to work with. I will need to reorganize the data into a more usable format.

In [133]:
import os
import pandas as pd

data = pd.read_csv(os.path.join('archive', 'pairs.csv'))
data.reset_index(drop=True, inplace=True)

data.columns = ['name1', 'image1', 'name2/image2', 'image2']

data.head()

,name1,image1,name2/image2,image2
0,Abel_Pacheco,1,4,NaN
1,Akhmed_Zakayev,1,3,NaN
2,Akhmed_Zakayev,2,3,NaN
3,Amber_Tamblyn,1,2,NaN
4,Anders_Fogh_Rasmussen,1,3,NaN


In [134]:
def castable_to_int(x):
    try:
        int(x)
        return True
    except:
        return False

data['image1'] = data['image1'].apply(lambda x: f"{int(x):04d}")
data['name2/image2'] = data['name2/image2'].apply(lambda x: f"{int(x):04d}" if castable_to_int(x) else x)
data['image2'] = data['image2'].apply(lambda x: f"{int(x):04d}" if pd.notna(x) else x)

data.head()

,name1,image1,name2/image2,image2
0,Abel_Pacheco,0001,0004,NaN
1,Akhmed_Zakayev,0001,0003,NaN
2,Akhmed_Zakayev,0002,0003,NaN
3,Amber_Tamblyn,0001,0002,NaN
4,Anders_Fogh_Rasmussen,0001,0003,NaN


Right now, the 3rd column is either the id of the image if it's a pair or the name of the other image if it's not a pair. I will reorganize the data so that the first column is the path to the first image, the second column is the path to the second image and the third column is 1 if they are a pair and 0 if they are not.

In [135]:
data['img1'] = data['name1'] + '/' + data['name1'] + '_' + data['image1']
data['img2'] = data.apply(lambda row: row['name2/image2'] + '/' + row['name2/image2'] + '_' + row['image2'] if pd.notna(row['image2']) else 0, axis=1)
data['img2'] = data.apply(lambda row: row['name1'] + '/' + row['name1'] + '_' + row['name2/image2'] if pd.isna(row['image2']) else row['img2'], axis=1)
data['pair'] = data.apply(lambda row: 1 if pd.notna(row['image2']) else 0, axis=1)

data.drop(['name1', 'image1', 'name2/image2', 'image2'], axis=1, inplace=True)
data

,img1,img2,pair
0,Abel_Pacheco/Abel_Pacheco_0001,Abel_Pacheco/Abel_Pacheco_0004,0
1,Akhmed_Zakayev/Akhmed_Zakayev_0001,Akhmed_Zakayev/Akhmed_Zakayev_0003,0
2,Akhmed_Zakayev/Akhmed_Zakayev_0002,Akhmed_Zakayev/Akhmed_Zakayev_0003,0
3,Amber_Tamblyn/Amber_Tamblyn_0001,Amber_Tamblyn/Amber_Tamblyn_0002,0
4,Anders_Fogh_Rasmussen/Anders_Fogh_Rasmussen_0001,Anders_Fogh_Rasmussen/Anders_Fogh_Rasmussen_0003,0
...,...,...,...
5995,Scott_Wolf/Scott_Wolf_0002,Troy_Polamalu/Troy_Polamalu_0001,1
5996,Sergei_Alexandrovitch_Ordzhonikidze/Sergei_Ale...,Yolanda_King/Yolanda_King_0001,1
5997,Shane_Loux/Shane_Loux_0001,Val_Ackerman/Val_Ackerman_0001,1
5998,Shawn_Marion/Shawn_Marion_0001,Shirley_Jones/Shirley_Jones_0001,1


In [136]:
data.to_csv(os.path.join( 'data.csv'), index=False)